# Detecting Fraudulent Bank Notes

<img src="http://i.dailymail.co.uk/i/pix/2011/01/17/article-1347979-0CCE3852000005DC-804_468x247.jpg"/>

We've all been hired by a bank to use machine learning and data science to determine if the bank notes they receive are real or not.

They've gone ahead and done a lot of the work for us, converting a training sample of bank notes into wavelet transformed samples. Each row in our data will have the following data:

1. variance of Wavelet Transformed image (continuous) 
2. skewness of Wavelet Transformed image (continuous) 
3. curtosis of Wavelet Transformed image (continuous) 
4. entropy of image (continuous) 
5. class (integer) 

It's up to us to use SVMs to try to predict genuine/not genuine.

[UCI Dataset](https://archive.ics.uci.edu/ml/datasets/banknote+authentication#)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

In [ ]:
data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00267/data_banknote_authentication.txt",
                   names=["Var","Skew","Kurt","Entropy","Genuine"])

In [ ]:
data.head()

In [ ]:
X = data.ix[:,:-1]
y = data.ix[:,-1]

# Exercise begin!


1. Use Logistic regression to detect genuine vs nongenuine bank notes. How does it perform in and out of sample?
2. Use SVM classifier to detect genuine vs nongenuine bank notes. Incorporate CV through grid search if you would like. Discuss your findings and thoughts with a partner.



## Helpful Notes:
1. If you try to fit and test on the entire dataset, you might get a 100% score.
2. Try using cross validation to see the performance out of sample
3. If this still confuses you, don't use all the features and/or try using a linear kernel.
4. Try plotting scatters of the features to see why (i.e. maybe there's already a linear hyperplane we can use without needing to use kernels). The scatters might shed light onto why a linear kernel might have a harder time.